In [ ]:
#ignore me, test setup

en1,x1,y1,z1 = rand(Float64,4)
en2,x2,y2,z2 = rand(Float32,4)
dot = en1*en2 - x1*x2 - y1*y2 - z1*z2



Leptonic Event Generator - a simple Julia HEP Project
========================================================


The best way to learn a programming language is by doing, so we're going to build a simple event generator - generating $e^+e^-\rightarrow\mu^+\mu^-$ events - in the `Julia` language.

The full completed source code is also available, but in these notebooks, we're going to guide you through the process file by file.


Our full project consists of one main Julia source file (*HEPExampleProject.jl*) plus subsiduary files divided strictly by their domain of application (*constants.jl*, *cross-section.jl*, *events.jl*, *four_momentum.jl*, *plotting.jl* and *utils.jl*).

We're not going to implement the main file, since it just ties everything else together into a package. Instead, we're going to implement the functionality of the other files, approximately in the order given.


First, let's look at **constants.jl** - this file exists purely to define `const` names so we can use them in the rest of the code.

As an aside, normally we'd not bother defining our own fundamental constants for commonly-known values - there's a very good Julia package called `PhysicalConstants.jl` that provides the usual constants we care about, and which interoperates with other packages like `Unitful.jl` to provide full support for unit conversion (and values with attached units). 
We're doing so here because it's a trivial example of writing Julia code that just declares some names with values.

We declare constants in Julia using the `const` keyword. Ideally, in order to allow Julia to generate code as efficiently as possible, we should also specify the *type* of the constant - however, for this example we will elide this unless you want to.

Specify the physical constants we need for the rest of the code, as given in the comments in **constants.jl**. We've provided one (ELECTRON_MASS, in MeV) for you.

In [5]:
const ELECTRON_MASS = 0.51099895000 #MeV
#MUON_MASS
#ALPHA

0.51099895

The first piece of non-trivial Julia code we need to complete is in **cross_section.jl**. 

We've left in the comments that describe the functions we need - to calculate the differential cross-section for a given energy and wrt scattering angle; and then to compute the total cross-section for a given energy (integrated over all scattering angles) - and the "prototypes" of the function, so we simply need to fill in the bodies.

First, to make our lives easier, we should define a utility function - `_rho` (where the _ indicates that this is likely a local function that isn't going to be exported) - which takes in a total relativistic energy, and a rest-mass, and returns the magnitude of the three-momentum.

We know from Special relativity that the total relativistic energy $E$ of a particle is related to the three-momentum $\rho$ and the rest-mass $m$ by:

$E^2 = m^2 + \rho^2$, where we're using reduced units so $c=1$.

Write a function `_rho` that takes in $E$ and $m$, and returns the value of $\rho$. 
(Remember, you can write a short simple function with the formalism:
functionname(args) = expression 
)
The assert statement in the frame below will test if your implementation is correct.

In [2]:
#_rho

_rho (generic function with 1 method)

In [6]:
@assert _rho(1e3, ELECTRON_MASS) == 999.999869440028

Note: we could also "decorate" this simple function with the @inline macro - just adding it to the start of the line with the function declaration. This would instruct the Julia JIT to expand out the function wherever it is called, rather than potentially calling it and incurring more runtime costs. 

Differential Cross Section

The doc-string for the differential cross section function tells us that the equation we need to implement is:

$\frac{\mathrm{d}\sigma}{\mathrm{d}\Omega} = \frac{\alpha^2}{16 E_{\mathrm{in}}^6}\left( E_{\text{in}}^4 + \rho_e^2 \rho_\mu^2 \cos^2\theta + E_{\text{in}}^2 \left( m_e^2 + m_\mu^2 \right) \right)
$

where:
- $E_{\text{in}}$ is the energy of the incoming electron/positron in CMS,
- $\rho_i$ are the magnitude of three-momenta of the electron ($i=e$) and muon ($i=\mu$) in the center-of-mass frame,
- $\alpha$ is the fine-structure constant,
- $m_e$ and $m_\mu$ are the masses of the electron and muon, respectively.

We already have a function that does $\rho_i$ for us.

Our task is to write a function that takes in $E_{\text{in}}$ and $\cos\theta$ and returns the value of the above expression. As this is going to need several lines, we're using the "long" form of a function declaration which starts

**function name(args)**

and ends

**end**

(You probably want to calculate $\rho$ for the electron and muon first - as this is in the CoM frame, energy conservation ensures that $E_{\text{in}} == E_{\text{out}}$ )

In [ ]:
function differential_cross_section(E_in,cos_theta)
    # fill me in

end

In [7]:
@testset "differential cross section" begin
    @testset "E = $E, cth = $cth" for (E,cth) in Iterators.product(ENERGIES, COSTHETAS)
        @test isapprox(differential_cross_section(E,cth),groundtruth_diffCS(E,cth))
    end
end

LoadError: LoadError: UndefVarError: `@testset` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at c:\Users\SamS\HEPExampleProject.jl\notebooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:1

Similarly, we can implement the total cross section for $e^+e^-\rightarrow\mu^+\mu^-$ at a given CoM energy, using the integral of the differential cross-section wrt $\theta$:

$\mathrm{d}\sigma(E_in) = \frac{\pi \alpha^2}{8 E_{\text{in}}^6} \cdot \frac{\rho_\mu}{\rho_e} \left( 2 E_{\text{in}}^4 + \frac{2}{3} \rho_\mu^2 \rho_e^2 + 2 E_{\text{in}}^2 (m_\mu^2 + m_e^2) \right)$


In [ ]:
function total_cross_section(E_in)
    #fill me in
end

In [ ]:
#assert

This completes the code needed to implement **cross_section.jl**.

### four_momentum.jl

In order to properly represent Events, we need a type that can represent the 4-Momentum of a particle. 4-Momenta are not simply vectors or tuples of 4 values: they have an algebra defined for them. Implementing a 4-Momentum type allow us to express this naturally via the standard Julia operators.

A natural construct to hold a 4-momentum would be a struct, with fields 
- `en`: Energy component.
- `x`: Spatial component in the x-direction.
- `y`: Spatial component in the y-direction.
- `z`: Spatial component in the z-direction.

However, it isn't clear what *type* these elements should have, other than that clearly they should all have the same type in the same instance of a 4-momentum. 
Julia supports us here, in that it allows us to create *parameterised* composite types.

In this case, we want all of the fields to have the same type, `T`, and for that type to be a parameter of the struct itself.
(Optionally, we can also constrain the set acceptable types for T, by writing a type identity or subset relation for T in the struct definition).


In [ ]:
struct FourMomentum{T}   #optionally - add a type constraint here so T can only be of typeclass Real
#elements of struct here, with correct type
end

# type promotion on construction 
#(this ensures that a FourMomentum built with mixed types is built with the most precise of those types)
# the ... operator expands a list into a set of parameters for a function
FourMomentum(en,x,y,z) = FourMomentum(promote(en,x,y,z)...)

In [ ]:
#this implements the isapprox relation for FourMomentum (floating-point aware approximate equality)
function Base.isapprox(
    p1::FourMomentum,
    p2::FourMomentum;
    atol::Real=0,
    rtol::Real=Base.rtoldefault(p1.x, p1.y, atol),
    nans::Bool=false,
    norm::Function=abs,
)
    return isapprox(p1.en, p2.en; atol=atol, rtol=rtol, nans=nans, norm=norm) &&
           isapprox(p1.x, p2.x; atol=atol, rtol=rtol, nans=nans, norm=norm) &&
           isapprox(p1.y, p2.y; atol=atol, rtol=rtol, nans=nans, norm=norm) &&
           isapprox(p1.z, p2.z; atol=atol, rtol=rtol, nans=nans, norm=norm)
end

Note: by default, structs are *immutable* types, so we can't edit the contents of a FourMomentum we've created.

However, we can create new ones via arithmetic operations, so we should extend the standard operators to do the right thing in these cases.
Julia uses *multiple-dispatch* for all function calls, so extending an operator is the same as defining that operator for a new combination of types on all its parameters.

The first thing we'll implement is something that's harder to express in languages where *types* themselves are not first-class elements - the `eltype` operation, which returns the type of an element when given the type of a container. 
(For example:
eltype(Array{Float64}) gives Float64
)

Because `eltype` is defined in another module (in this case, the `Base` module that contains the fundamental Julia language primitives), we need to specify its name fully - including the Module name - if we're going to add methods to it for a new type.

A trick with eltype in particular is that we can use "generic" type parameters as arguments in our function - and you don't *have* to name a parameter if you're not actually going to use it in your function.
So, here, we only care that the parameter is of type FourMomentum{T} - and thus has elements of type T - we don't care about the actual values at all.

In [ ]:
Base.eltype(::FourMomentum{T}) 
#finish this off - add a where to restrict T (are there any?) and then the function implementation itself
#                   which is very very short!

In [ ]:
p1 = FourMomentum(en1,x1,y1,z1)
p2 = FourMomentum(en2,x2,y2,z2)
@assert eltype(p1) == Float64 && eltype(p2) == Float32


Next let's define the basic arithmetic operations for FourMomentum vector. Again, these operators are defined in the Base module.

For addition and subtraction, these are simply element-wise application of the same operator across the pair of FourMomenta - we can do this "inside" the constructor for the new FourMomentum to save space and allocations

In [ ]:
function Base.:+(p1::FourMomentum, p2::FourMomentum) 
    
end

#do subtraction as well


In [ ]:
p1_plus_2 = p1 + p2
@assert eltype(p1_plus_p2)==Float64 && isapprox(p1_plus_p2.en,en1 + en2) && isapprox(p1_plus_p2.x,x1 + x2) && isapprox(p1_plus_p2.y,y1 + y2) && isapprox(p1_plus_p2.z,z1 + z2)

For multiplication by a scalar, this is also just element-wise application of the scalar across all the fields.

In [ ]:
function Base.:*(a::Real, p::FourMomentum) 
    
end

Finally, for the dot product (with the Minkowski metric), the result is the sum over the paired elements, with signs given by the metric.

In [ ]:
function minkowski_dot(#fill me in)
    # Minkowski metric: (+,-,-,-)
    
end

In [ ]:
p1_times_p2 = minkowski_dot(p1,p2)
@assert isapprox(p1_times_p2,dot)

We also provide some utility functions for creating FourMomenta from other parameters which fully specify the context of an event.

Given a CoM energy, $E_{in}$, and the scattering angles $\cos\theta$ and $\phi$, we can generate a prototypical event by assuming the $e^+e^-$ collision is colinear with the $z$-axis, and then rotating the $\mu^+\mu^-$ components relative to them. 
As always, we can use the `rho` function we defined first to get the magnitude of the 3-momentum from the energy, for a given particle mass.

(To save you remembering the mathematics, with $z$ as the principal axis, the components in the $x,y,z$ axes will be $\sin\theta\cos\phi, \sin\theta\sin\phi, \cos\theta$ respectively)

The "event" we generate here is thus simply a tuple of 4 FourMomenta, in the order $e^-$,$e^+$,$\mu^-$,$\mu^+$... 

In [ ]:
function _construct_moms_from_coords(E_in,cos_theta,phi)
    #implement me
end